# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [ ]:
#create a master cities dict 
CitiesDict = {'New York' : 'New York City', 
             'New Jersey' : 'New York City', 
             'Brooklyn' : 'New York City',
             'Anaheim' : 'Los Angeles',
             'Los Angeles' : 'Los Angeles', 
             'San Francisco' : 'San Francisco Bay Area',
             'San Jose' : 'San Francisco Bay Area', 
             'Golden' : 'San Francisco Bay Area', 
             'Oakland' : 'San Francisco Bay Area', 
             'Sacramento' : 'Sacramento',
             'San Diego' : 'San Diego',
             'Dallas' : 'Dallas–Fort Worth', 
             'Texas Rangers' : 'Dallas–Fort Worth',
             'Washington' : 'Washington, D.C.', 
             'Minnesota' : 'Minneapolis–Saint Paul', 
             'Colorado' : 'Denver', 
             'Denver' : 'Denver',
             'Florida' : 'Miami–Fort Lauderdale', 
             'Miami Heat' : 'Miami–Fort Lauderdale',
             'Miami Marlins' : 'Miami–Fort Lauderdale',
             'Miami Dolphins' : 'Miami–Fort Lauderdale',
             'Arizona' : 'Phoenix', 
             'Phoenix' : 'Phoenix',
             'Tampa Bay' : 'Tampa Bay Area', 
             'Carolina' : 'Raleigh',
             'Vegas Golden' : 'Las Vegas', 
             'Boston' : 'Boston', 
             'New England' : 'Boston',
             'Toronto' : 'Toronto', 
             'Detroit' : 'Detroit', 
             'Montreal' : 'Montreal', 
             'Ottawa' : 'Ottawa', 
             'Buffalo' : 'Buffalo', 
             'Pittsburgh' : 'Pittsburgh', 
             'Philadelphia' : 'Philadelphia', 
             'Columbus' : 'Columbus', 
             'Nashville' : 'Nashville', 
             'Tennessee' : 'Nashville',
             'Winnipeg' : 'Winnipeg', 
             'St. Louis' : 'St. Louis', 
             'Chicago' : 'Chicago', 
             'Calgary' : 'Calgary', 
             'Edmonton' : 'Edmonton', 
             'Vancouver' : 'Vancouver', 
             'Cleveland' : 'Cleveland', 
             'Indiana' : 'Indianapolis', 
             'Indianapolis' : 'Indianapolis', 
             'Milwaukee' : 'Milwaukee', 
             'Charlotte' : 'Charlotte', 
             'Orlando' : 'Orlando', 
             'Atlanta' : 'Atlanta', 
             'Houston' : 'Houston', 
             'Portland' : 'Portland', 
             'Oklahoma' : 'Oklahoma City', 
             'Utah Jazz' : 'Salt Lake City', 
             'New Orleans' : 'New Orleans', 
             'San Antonio' : 'San Antonio', 
             'Memphis' : 'Memphis', 
             'Baltimore' : 'Baltimore', 
             'Kansas' : 'Kansas City', 
             'Seattle' : 'Seattle', 
             'Cincinnati' : 'Cincinnati', 
             'Green Bay' : 'Green Bay', 
             'Jacksonville' : 'Jacksonville'}

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [ ]:
def get_nhl_df():
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    #add win/loss ratio column to nhl_df
    nhl_df.set_index('year', inplace=True)
    nhl_df=nhl_df.loc[2018] #only keep 2018
    nhl_df.reset_index(inplace=True)
    nhl_df.drop(index=[0,9,18,26], axis=1, inplace=True)
    nhl_df['team'] = nhl_df['team'].str.replace(r"\*","") #got rid of the stars 
    nhl_df['W'] = nhl_df['W'].astype(int) #turn to an integer
    nhl_df['L'] = nhl_df['L'].astype(int)
    nhl_df['W/L%'] = nhl_df['W']/(nhl_df['W'] + nhl_df['L'])
    
    #separate city from team 
    nhl_df['city']=nhl_df['team'].str.extract(r'^([\w.]{1,5}(?:\s\w+)?\w*)')
    nhl_df.set_index('city', inplace=True)

    #use dict for city fixing to align with cities df
    nhl_df['Metropolitan area'] = pd.Series(CitiesDict)

    #keep only necessary info
    nhl_df = nhl_df[['year', 'team', 'W', 'L', 'W/L%', 'Metropolitan area']]
    nhl_df.reset_index(inplace=True)
    nhl_df.drop(columns=['city'], inplace=True)

    #merge on the metro areas to assure things are in the same order
    df1 = pd.merge(nhl_df, cities, how='inner', on='Metropolitan area')
    df1.sort_values('Metropolitan area', ascending=True, inplace=True)
    df1.reset_index(inplace=True)
    df1.drop(columns=['index', 'NFL', 'MLB', 'NBA', 'NHL'],inplace=True)
    
    #merge together the same metro areas
    df1['Population (2016 est.)[8]'] = df1['Population (2016 est.)[8]'].astype(float)
    groupdf1=df1.groupby('Metropolitan area').agg({'Population (2016 est.)[8]':np.nanmean, 'W/L%' : np.nanmean})
    
    return groupdf1
get_nhl_df()

In [ ]:
def nhl_correlation(): 
    groupdf1 = get_nhl_df()
    

    population_by_region = groupdf1['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = groupdf1['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

nhl_correlation()

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
def get_nba_df():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    #add win/loss ratio column to nba_df
    nba_df.set_index('year', inplace=True)
    nba_df=nba_df.loc[2018] #only keep 2018
    nba_df.reset_index(inplace=True)
    nba_df['team'] = nba_df['team'].str.replace(r"\*\s\(.*\)","") #got rid of the stars and parentheses with numbers
    nba_df['team'] = nba_df['team'].str.replace(r"\s\(.*\)","")
    nba_df['W'] = nba_df['W'].astype(int) #turn to an integer
    nba_df['L'] = nba_df['L'].astype(int)
    nba_df['W/L%'] = nba_df['W']/(nba_df['W'] + nba_df['L'])
    
    #separate city from team 
    nba_df['city']=nba_df['team'].str.extract(r'^([\w.]{1,5}(?:\s\w+)?\w*)')
    nba_df.set_index('city', inplace=True)

    #use dict for city fixing to align with cities df
    nba_df['Metropolitan area'] = pd.Series(CitiesDict)
    
    #keep only necessary info
    nba_df = nba_df[['year', 'team', 'W', 'L', 'W/L%', 'Metropolitan area']]
    nba_df.reset_index(inplace=True)
    nba_df.drop(columns=['city'], inplace=True)

    #merge on the metro areas to assure things are in the same order
    df2 = pd.merge(nba_df, cities, how='inner', on='Metropolitan area')
    df2.sort_values('Metropolitan area', ascending=True, inplace=True)
    df2.reset_index(inplace=True)
    df2.drop(columns=['index', 'NFL', 'MLB', 'NBA', 'NHL'],inplace=True)
    
    #merge together the same metro areas
    df2['Population (2016 est.)[8]'] = df2['Population (2016 est.)[8]'].astype(float)
    groupdf2=df2.groupby('Metropolitan area').agg({'Population (2016 est.)[8]':np.nanmean, 'W/L%' : np.nanmean})
    
    return groupdf2

In [ ]:
def nba_correlation():
    groupdf2 = get_nba_df()
    
    population_by_region = groupdf2['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = groupdf2['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

nba_correlation()

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
def get_mlb_df(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    #add win/loss ratio column to mlb_df
    mlb_df.set_index('year', inplace=True)
    mlb_df=mlb_df.loc[2018] #only keep 2018
    mlb_df.reset_index(inplace=True)
    mlb_df['W'] = mlb_df['W'].astype(int) #turn to an integer
    mlb_df['L'] = mlb_df['L'].astype(int)
    mlb_df['W/L%'] = mlb_df['W']/(mlb_df['W'] + mlb_df['L'])
    
    #separate city from team 
    mlb_df['city']=mlb_df['team'].str.extract(r'^([\w.]{1,5}(?:\s\w+)?\w*)')
    mlb_df.set_index('city', inplace=True)

    #use dict for city fixing to align with cities df
    mlb_df['Metropolitan area'] = pd.Series(CitiesDict)
    
    #keep only necessary info
    mlb_df = mlb_df[['year', 'team', 'W', 'L', 'W/L%', 'Metropolitan area']]
    mlb_df.reset_index(inplace=True)
    mlb_df.drop(columns=['city'], inplace=True)

    #merge on the metro areas to assure things are in the same order
    df3 = pd.merge(mlb_df, cities, how='inner', on='Metropolitan area')
    df3.sort_values('Metropolitan area', ascending=True, inplace=True)
    df3.reset_index(inplace=True)
    df3.drop(columns=['index', 'NFL', 'MLB', 'NBA', 'NHL'],inplace=True)
    
    #merge together the same metro areas
    df3['Population (2016 est.)[8]'] = df3['Population (2016 est.)[8]'].astype(float)
    groupdf3=df3.groupby('Metropolitan area').agg({'Population (2016 est.)[8]':np.nanmean, 'W/L%' : np.nanmean})
    
    return groupdf3

In [ ]:
def mlb_correlation(): 
    groupdf3 = get_mlb_df()
    
    population_by_region = groupdf3['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = groupdf3['W/L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

mlb_correlation()

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
def get_nfl_df(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    #add win/loss ratio column to nhl_df
    nfl_df.set_index('year', inplace=True)
    nfl_df=nfl_df.loc[2018] #only keep 2018
    nfl_df.reset_index(inplace=True)
    nfl_df.drop(index=[0,5,10,15,20,25,30,35], axis=1, inplace=True)
    nfl_df['team'] = nfl_df['team'].str.replace(r"\*","") #got rid of the stars 
    nfl_df['team'] = nfl_df['team'].str.replace(r"\+","") #got rid of the pluses
    nfl_df['W'] = nfl_df['W'].astype(int) #turn to an integer
    nfl_df['L'] = nfl_df['L'].astype(int)
    nfl_df['W/L%'] = nfl_df['W']/(nfl_df['W'] + nfl_df['L'])
    
    #separate city from team 
    nfl_df['city']=nfl_df['team'].str.extract(r'^([\w.]{1,5}(?:\s\w+)?\w*)')
    nfl_df.set_index('city', inplace=True)

    #use dict for city fixing to align with cities df
    nfl_df['Metropolitan area'] = pd.Series(CitiesDict)
    nfl_df.set_value('Carolina', 'Metropolitan area', 'Charlotte')

    #keep only necessary info
    nfl_df = nfl_df[['year', 'team', 'W', 'L', 'W/L%', 'Metropolitan area']]
    nfl_df.reset_index(inplace=True)
    nfl_df.drop(columns=['city'], inplace=True)

    #merge on the metro areas to assure things are in the same order
    df4 = pd.merge(nfl_df, cities, how='inner', on='Metropolitan area')
    df4.sort_values('Metropolitan area', ascending=True, inplace=True)
    df4.reset_index(inplace=True)
    df4.drop(columns=['index', 'NFL', 'MLB', 'NBA', 'NHL'],inplace=True)
    
    #merge together the same metro areas
    df4['Population (2016 est.)[8]'] = df4['Population (2016 est.)[8]'].astype(float)
    groupdf4=df4.groupby('Metropolitan area').agg({'Population (2016 est.)[8]':np.nanmean, 'W/L%' : np.nanmean})
    
    return groupdf4   

In [ ]:
def nfl_correlation(): 
    groupdf4 = get_nfl_df()
    
    population_by_region = groupdf4['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = groupdf4['W/L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

nfl_correlation()

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
#mlb_df=pd.read_csv("assets/mlb.csv")
#nhl_df=pd.read_csv("assets/nhl.csv")
#nba_df=pd.read_csv("assets/nba.csv")
#nfl_df=pd.read_csv("assets/nfl.csv")
#cities=pd.read_html("assets/wikipedia_data.html")[1]
#cities=cities.iloc[:-1,[0,3,5,6,7,8]]


# Let's bring in our ttest_ind function
from scipy.stats import ttest_ind

def sports_team_performance():
    mlb_df = get_mlb_df()
    nhl_df = get_nhl_df()
    nba_df = get_nba_df()
    nfl_df = get_nfl_df()
    
    mlb_df.reset_index(inplace = True)
    nhl_df.reset_index(inplace = True)
    nba_df.reset_index(inplace = True)
    nfl_df.reset_index(inplace = True)
    
    df1 = pd.merge(mlb_df, nhl_df, how='outer', on=['Metropolitan area', 'Population (2016 est.)[8]'])
    df1.rename(columns = {'W/L%_x':'W/L% MLB', 'W/L%_y':'W/L% NHL'}, inplace = True)
    df2 = pd.merge(nba_df, nfl_df, how='outer', on=['Metropolitan area', 'Population (2016 est.)[8]'])
    df2.rename(columns = {'W/L%_x':'W/L% NBA', 'W/L%_y':'W/L% NFL'}, inplace = True)
    df = pd.merge(df1,df2, how = 'outer', on=['Metropolitan area', 'Population (2016 est.)[8]'])
    df.rename(columns={'Population (2016 est.)[8]' : 'Population'}, inplace=True)
    
    
    #MLB and NHL
    mlb_nhl = df.drop(columns=['W/L% NBA', 'W/L% NFL'])
    mlb_nhl.dropna(inplace=True)
    stat_mlb_nhl, pval_mlb_nhl = ttest_ind(mlb_nhl['W/L% MLB'], mlb_nhl['W/L% NHL'])
    
    #MLB and NBA
    mlb_nba = df.drop(columns=['W/L% NHL', 'W/L% NFL'])
    mlb_nba.dropna(inplace=True)
    stat_mlb_nba, pval_mlb_nba = ttest_ind(mlb_nba['W/L% MLB'], mlb_nba['W/L% NBA'])
    
    #MLB and NFL
    mlb_nfl = df.drop(columns=['W/L% NBA', 'W/L% NHL'])
    mlb_nfl.dropna(inplace=True)
    stat_mlb_nfl, pval_mlb_nfl = ttest_ind(mlb_nfl['W/L% MLB'], mlb_nfl['W/L% NFL'])
    
    #NHL and NBA
    nhl_nba = df.drop(columns=['W/L% MLB', 'W/L% NFL'])
    nhl_nba.dropna(inplace=True)
    stat_nhl_nba, pval_nhl_nba = ttest_ind(nhl_nba['W/L% NHL'], nhl_nba['W/L% NBA'])
    
    #NHL and NFL
    nhl_nfl = df.drop(columns=['W/L% MLB', 'W/L% NBA'])
    nhl_nfl.dropna(inplace=True)
    stat_nhl_nfl, pval_nhl_nfl = ttest_ind(nhl_nfl['W/L% NHL'], nhl_nfl['W/L% NFL'])
    
    #NBA and NFL
    nba_nfl = df.drop(columns=['W/L% MLB', 'W/L% NHL'])
    nba_nfl.dropna(inplace=True)
    stat_nba_nfl, pval_nba_nfl = ttest_ind(nba_nfl['W/L% NBA'], nba_nfl['W/L% NFL'])
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    #put necessary values into spots in p_values
    p_values.at['NFL', 'NBA'] = pval_nba_nfl
    p_values.at['NBA', 'NFL'] = pval_nba_nfl
    
    p_values.at['NFL', 'NHL'] = pval_nhl_nfl
    p_values.at['NHL', 'NFL'] = pval_nhl_nfl
    
    p_values.at['NFL', 'MLB'] = pval_mlb_nfl
    p_values.at['MLB', 'NFL'] = pval_mlb_nfl
    
    p_values.at['NBA', 'NHL'] = pval_nhl_nba
    p_values.at['NHL', 'NBA'] = pval_nhl_nba
    
    p_values.at['NBA', 'MLB'] = pval_mlb_nba
    p_values.at['MLB', 'NBA'] = pval_mlb_nba
    
    p_values.at['NHL', 'MLB'] = pval_mlb_nhl
    p_values.at['MLB', 'NHL'] = pval_mlb_nhl
    
    
    #assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    #assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

sports_team_performance()